<a href="https://colab.research.google.com/github/jzheng23/colab/blob/main/Screen_time_sign_up_data_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Jian
google_drive_data_path = '/content/drive/MyDrive/Problematic smartphone usage/Ambient display/Data'
google_drive_key_path = '/content/drive/MyDrive/Problematic smartphone usage/Ambient display/Key'

### Read and save the raw data

In [ ]:
import pandas as pd
import requests
import zipfile
import json
import time
import io

def get_qualtrics_data(api_token, survey_id):
   base_url = f"https://pdx1.qualtrics.com/API/v3/surveys/{survey_id}/export-responses"
   headers = {
       "X-API-TOKEN": api_token,
       "Content-Type": "application/json"
   }

   # Start export and get progress ID
   export_response = requests.post(base_url, json={"format": "csv"}, headers=headers)
   export_progress_id = export_response.json()["result"]["progressId"]

   # Poll until complete
   while True:
       progress_response = requests.get(f"{base_url}/{export_progress_id}", headers=headers)
       if progress_response.json()["result"]["status"] == "complete":
           file_id = progress_response.json()["result"]["fileId"]
           break
       time.sleep(1)

   # Download and extract
   download_response = requests.get(f"{base_url}/{file_id}/file", headers=headers)
   with zipfile.ZipFile(io.BytesIO(download_response.content)) as zf:
       return zf.read(zf.namelist()[0]).decode('utf-8')

sign_up_survey = "SV_3RiDob4GtY8kCSG"
api_token = "U5xGlZmJv76LsjIXvfwB7FS9RqrqwmMb3vva3pbD"

# Get data and save
signup_survey_data = get_qualtrics_data(api_token, sign_up_survey)
signup_df = pd.read_csv(io.StringIO(signup_survey_data))
print(f"The shape of the signup_survey_data is {signup_df.shape}")
signup_df.to_csv(google_drive_data_path + '/signup_survey_data_raw.csv', index=False)

The shape of the signup_survey_data is (678, 55)


### Process the data

In [ ]:
# install the package if missing
!pip install reverse_geocoder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for reverse_geocoder: filename=reverse_geocoder-1.5.1-py3-none-any.whl size=2268068 sha256=b4e1886df2941f11707a87e9e7f3470fb0b9ff4f1cc2775d8e81fb607a395ad1
  Stored in directory: /root/.cache/pip/wheels/bd/e5/88/eb139b6d6a26b8022d370ab991f7a836802fed9871975ec6d9
Successfully built reverse_geocoder


In [ ]:
import pandas as pd
import reverse_geocoder as rg

def get_country(lat, lon):
   try:
       result = rg.search((lat, lon))[0]['cc']
       return result if result else 'Unknown'
   except:
       return 'Unknown'

# Load and preprocess data
signup_data = pd.read_csv(google_drive_data_path + '/signup_survey_data_raw.csv').drop([0, 1])
signup_data['Q19'] = signup_data['Q19'].replace('djsboca@aol.ocm', 'djsboca@aol.com')

# Add country info
signup_data['Country'] = signup_data.apply(
   lambda row: get_country(row['LocationLatitude'], row['LocationLongitude']), axis=1
)

# Drop missing Q19 and init valid column
signup_data = signup_data.dropna(subset=['Q19'])
signup_data['valid'] = 1

# Add location and IP counts
signup_data['location_count'] = signup_data.groupby(['LocationLatitude', 'LocationLongitude'])['LocationLatitude'].transform('count')
signup_data['ip_count'] = signup_data.groupby('IPAddress')['IPAddress'].transform('count')

# Update valid flags
conditions = [
   (signup_data['location_count'] != 1, 9),
   (signup_data['Country'] != 'US', 2),
   (signup_data['ip_count'] != 1, 3),
   ((signup_data['Q_RecaptchaScore'].astype(float) < 0.8) | (signup_data['Q_RelevantIDDuplicate'] == "TRUE"), 5),
   (signup_data['Q_RelevantIDFraudScore'].astype(float) > 0, 5),
   (signup_data['Q23'].str.contains('jan|rana|wechat|326|class', case=False, na=False), 1),
   (~signup_data['Q19'].str.lower().str.endswith('@gmail.com', na=False), 1),
   (signup_data['Q23'].str.contains('Facebook|reddit|flyer', case=False, na=False), 4)
]

for condition, value in conditions:
   signup_data.loc[condition, 'valid'] = value

# Remove Q19 duplicates and save
signup_data = signup_data.drop_duplicates(subset=['Q19'], keep='first')
signup_data.to_csv(google_drive_data_path + '/signup_survey_data_labeled.csv', index=False)

Loading formatted geocoded file...


<ipython-input-5-8ab112c712cc>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signup_data['valid'] = 1
<ipython-input-5-8ab112c712cc>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signup_data['location_count'] = signup_data.groupby(['LocationLatitude', 'LocationLongitude'])['LocationLatitude'].transform('count')
<ipython-input-5-8ab112c712cc>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

Get city from location

In [ ]:
signup_data0 = pd.read_csv(google_drive_data_path + '/signup_survey_data_labeled.csv')
# Create a dictionary from the CSV mapping
column_mapping = {
    'EndDate': 'EndDate',
    'Q3': 'Age',
    'Q5': 'Brand',
    'Q5_5_TEXT': 'Brand_text',
    'Q6': 'Android_version',
    'Q11': 'Screen_time',
    'Q13': 'Gender',
    'Q13_4_TEXT': 'Gender_text',
    'Q14': 'Education',
    'Q15_1': 'Race_white',
    'Q15_2': 'Race_black',
    'Q15_3': 'Race_native',
    'Q15_4': 'Race_Asian',
    'Q15_5': 'Race_Pacific',
    'Q15_6': 'Race_others',
    'Q15_7': 'Race_text',
    'Q16_1': 'Employment_retired',
    'Q16_2': 'Employment_self-employed',
    'Q16_3': 'Employment_employment full-time',
    'Q16_4': 'Employment_employment part-time',
    'Q16_5': 'Employment_voluntary worker',
    'Q16_6': 'Employment_homemaker',
    'Q16_7': 'Employment_student',
    'Q16_8': 'Employment_others',
    'Q16_4_TEXT': 'Employment_hours_text',
    'Q16_8_TEXT': 'Employment_text',
    'Q18': 'Name',
    'Q19': 'Email',
    'Q23': 'Source'
}

# Filter valid rows and rename columns
signup_data0 = (signup_data0[signup_data0['valid'].isin([1, 9])]
               [column_mapping.keys()]
               .rename(columns=column_mapping))

# Save to CSV
signup_data0.to_csv(google_drive_data_path + '/signup_survey_data_valid.csv', index=False)

In [ ]:
def process_signup_data(df):
    # Create a copy
    df = df.copy()

    # Define mappings
    mappings = {
        'Brand': {'1': 'Samsung', '3': 'Xiaomi', '4': 'Google'},
        'Gender': {'1': 'female', '2': 'male', '3': 'non-binary'},
        'Education': {
            '1': 'some high school or less',
            '2': 'high school or GED',
            '3': 'some college no degree',
            '4': 'associate or technical degree',
            '5': 'bachelor',
            '6': 'graduate or professional',
            '7': 'prefer not to say'
        }
    }

    # Apply mappings with fallback to text columns
    df['Brand'] = df.apply(lambda x: mappings['Brand'].get(str(x['Brand']), x['Brand_text']), axis=1)
    df['Gender'] = df.apply(lambda x: mappings['Gender'].get(str(x['Gender']), x['Gender_text']), axis=1)
    df['Education'] = df['Education'].astype(str).map(mappings['Education'])

    # Combine race columns
    race_cols = ['white', 'black', 'native', 'Asian', 'Pacific', 'others']
    df['Race'] = df.apply(lambda x: ', '.join(
        [race for race, col in zip(race_cols, ['Race_' + r for r in race_cols])
         if str(x[col]) == '1.0'] +
        ([x['Race_text']] if pd.notna(x['Race_text']) and x['Race_text'] else [])), axis=1)

    # Combine employment columns
    emp_mappings = {
        'Employment_retired': 'retired',
        'Employment_self-employed': 'self-employed',
        'Employment_employment full-time': 'full-time',
        'Employment_employment part-time': 'part-time',
        'Employment_voluntary worker': 'voluntary worker',
        'Employment_homemaker': 'homemaker',
        'Employment_student': 'student',
        'Employment_others': 'others'
    }

    df['Employment'] = df.apply(lambda x: ', '.join(
        [val for col, val in emp_mappings.items() if str(x[col]) == '1.0'] +
        ([x['Employment_text']] if pd.notna(x['Employment_text']) and x['Employment_text'] else [])), axis=1)

    # Remove "Android version" or "Android", together with space before or after
    df['Android_version'] = df['Android_version'].str.replace(r'\s*(Android version|Android)\s*', '', case=False, regex=True)

    # Remove "hours" or "hrs", together with space before or after
    df['Screen_time'] = df['Screen_time'].str.replace(r'\s*(hours|hrs)\s*', '', case=False, regex=True)

    # Rest of the code remains the same
    cols_to_drop = (['Brand_text', 'Gender_text', 'Race_text', 'Employment_text'] +
                    ['Race_' + r for r in race_cols] +
                    list(emp_mappings.keys()))
    df = df.drop(cols_to_drop, axis=1)
    df = df.rename(columns={'Employment_hours_text': 'Part_time_hours'})

    columns_order = [
        'EndDate', 'Age', 'Gender', 'Education', 'Race', 'Employment', 'Part_time_hours',
        'Brand', 'Android_version', 'Screen_time',
        'Name', 'Email', 'Source'
    ]
    return df[columns_order]

# Process data
signup_data1 = process_signup_data(signup_data0)
signup_data1['Email'] = signup_data1['Email'].str.lower().replace('djsboca@aol.ocm', 'djsboca@aol.com')
signup_data1.to_csv(google_drive_data_path + '/signup_survey_data_processed.csv', index=False)

In [ ]:
# prompt: in process_signup_data, remove "Android version" or "Android", together with space before or after the string, case insensitive, from the string in the Android_version column.  Remove "hours" or "hrs", together with space before or after the string, case insensitive, from the string in the Screen_time column.

def process_signup_data(df):
    # Create a copy
    df = df.copy()

    # ... (Existing code)

    # Remove "Android version" or "Android", together with space before or after
    df['Android_version'] = df['Android_version'].str.replace(r'\s*(Android version|Android)\s*', '', case=False, regex=True)

    # Remove "hours" or "hrs", together with space before or after
    df['Screen_time'] = df['Screen_time'].str.replace(r'\s*(hours|hrs)\s*', '', case=False, regex=True)

    # ... (Rest of the code)

Combine files

In [ ]:
# Read and process files
df1 = pd.read_csv(google_drive_data_path + '/signup_survey_data_processed.csv')
df2 = pd.read_excel(google_drive_data_path + '/signup_survey_data_processed_combined.xlsx')
participant_data = pd.read_excel(google_drive_data_path + '/Participants.xlsx')

# Convert all emails to lowercase
df1['Email'] = df1['Email'].str.lower()
df2['Email'] = df2['Email'].str.lower()
participant_data['Email'] = participant_data['Email'].str.lower()

# Process and combine
new_rows = df1[~df1['Email'].isin(df2['Email'])].assign(Recruit=2, Note='')
final_df = pd.concat([df2, new_rows], ignore_index=True)

if 'PID' in final_df.columns:
   final_df = final_df.drop('PID', axis=1)

# Merge PID and EndDate
final_df = (final_df
           .merge(participant_data[['Email','PID']], on='Email', how='left'))

print(f"The shape of the final_df is {final_df.shape}")
final_df.to_excel(google_drive_data_path + '/signup_survey_data_processed_combined.xlsx', index=False)

The shape of the final_df is (85, 16)


Create a mail list to upload

In [ ]:
participant_data = pd.read_excel(google_drive_data_path + '/Participants.xlsx')

In [ ]:
participant_data.tail()

,Email,nick name,PID,Date
41,vkurmalev@gmail.com,Vlad,ST050,2024-12-07
42,madiflori1@gmail.com,Madi,ST051,2024-12-07
43,finplan65@yahoo.com,Philip,ST052,2024-12-07
44,miller.jennylee@yahoo.com,Jen,ST053,2024-12-07
45,danmadkia@gmail.com,Nakia,ST054,2024-12-07


In [ ]:
# prompt: only keep those where Date (e.g., 2024-12-07) = Today in the participant_data above, then drop Date, save the df to a csv file with today.csv (e.g. 1207.csv)

import pandas as pd
from datetime import date

# Assuming participant_data is already loaded as a DataFrame
# participant_data = pd.read_excel(...)  # Replace with your actual loading code

today = date.today().strftime("%Y-%m-%d")
today_short = date.today().strftime("%m%d")

# Filter data for today's date
participant_data_today = participant_data[participant_data['Date'] == today]

# Drop the 'Date' column
participant_data_today = participant_data_today.drop('Date', axis=1, errors='ignore')

# Save to CSV
participant_data_today.to_csv(google_drive_data_path + f'/{today_short}.csv', index=False)